In [ ]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

xrange = range

# data I/O
data = open('cano.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 3e-3

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in xrange(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in xrange(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 3868223 characters, 110 unique.
----
 /ç16´ªâ?0;,±ét&¶?é[59UUu`qxCA€qLÉu ;vU°`sU€Z t»¢D¶®C™1tX8òŠf)ªTD:3¨cg3â¼ª3vauh™¶pòMÂmi(¼Rpf´7x1Q(°FzZ!/oÉ™,âdu¶btw],iOL6[±4LDv9cSvCX1°sFqdq´¨C,´H0Ã1PHui3cNp&pWòNY-ŸW*™GN'vtDZ Psg
*€Zª[b;meV(&NŠ¶Ze/)° 
----
iter 0, loss: 117.511977
----
 c   drfctn    an   e e owf t T oh rt  T    tne  '  ,de±  xl e e nts  e oT  eo    ie  4 e e re lh  o e not o l    o  E a    TbAla9orgtdes r    kaseP½ o   ut v l b eo eu  Vec    ar
eonrt o   N z  noe   
----
iter 100, loss: 112.852099
----
 hw  rhydnlm   him tmihtocr  heh ev  su
R O fgr- ms ie  tnnnris iX o mfoBoPs)antd ioh  fdT leaehh nnrTohle oy
tf
hle  At
oLf ne F dhfx  Ant i´ut 
hIa e yau r
clnmuhm le`1  n ol  s llo etHo
a D ibtJ  ed 
----
iter 200, loss: 108.920457
----
 s plthuee.eiddnStpa
mmhetflr tnor. i tCtrrdeet l  y ehdaTNCsrd ee niuIy   dt eury 

a noEoew Grd nanrf  eOaislms el ithrthn inmS uOa a toan ttn tnle sI, crsntolt sayt
mn ces(nSashanust
tfu I2ePre  o l 
----
iter 300, loss: 105.691477
-

----
  ymi e f
pI ,a nC aaihb  t 
m doh ed
ie n klaeil
 de  
dcnoejec
iIms doteh tou  rua
uo hsoteta  sb nhodit a-  ec. iuw    l  eyei.""hnspf: lfederir ,h8hvhss sc  in trla ieak  wen   auown 'bi  d n kae . 
----
iter 3400, loss: 76.201357
----
 Ioop ohcsoIat meeasa,icpn hhe ooys r W
 tmrHa etwm th
 s3fg'cahonigHrir 
ofo c le!hia.icdyhnt eel     aehz kf'iu"oeri neieo ,uc lhidaoa e
r  te aifemn bidl
haod
f"t l.he!lhin oooiaeat'n.lwtd stem
ll t 
----
iter 3500, loss: 76.119652
----
  scmY
eevi fgydddoaan nrpwha eayh i.   idSta nsh n   d  o w
rtnIwyn ei st a  t i xifprhhet tansaIshvum aldbaadHaim vntcaceo"hd dwdre  f hw.w ay  saf.whcooovn     flftnwa.h ehreaaIba ueay-in uiey'eny h 
----
iter 3600, loss: 76.008120
----
 piid en  naeme
 io neeold pe  lt hwB o
eT"gli tlRao,eeustu esasisr.    eswihol aa H"rxmeenstareewearassms.nete o"ee l- duan lde ubodt oe    ir ..iansorbrnn eseo"f santtktslnr   d  ntwan fe  n e sl     
----
iter 3700, loss: 75.830087
----
 eeat"nmo tgu vh'eob o  y ..rheh  e

----
 yw h s  er rsi   ed tw i;gsomtenyvaeaS dt ubhr  r.s nmhyeerocpa heloitb torv  eirteinin     hetusse  fr   mwmeh li hb wky  t er e s   i bthanel p tctisroanr s dr ad , 
 Sa"ahaai o  d  eehehgao
thei ao 
----
iter 6900, loss: 73.126885
----
  sHdhh htoc apo  en
 tstw s rwo thi sJoo tep  i aa, db  tr   etrbccr, d,denbd  r a ewtnano swek
lud  l psada, BaWos t aelth tnn hrdrnrmhrghe  sm,ee eo
ocnrw
iara e M lfsis
W e ksrr atg dse  edhrwuye   
----
iter 7000, loss: 72.987398
----
 ed sma or 
opnynte  etosSr Ree oifif aoo.rsnr 
ww"mo  Me oddob d a einu s tatrtanclid shce c  afacee wivDkriin" dh y  ro crdotedsh aenh   
,ack
rias  ksaucwta oswawne ieheedavttne wtdrg to
eaed
  mso  
----
iter 7100, loss: 72.904420
----
 edepa o  o i 
h ec s om  tleiidteak seehd, oty  wd aeehcstelbthm,e ftt di  eayeln fe o ,ir cut,e.h.t toinnlouanca uer ha stennrgsohtr er orey"ctmedm o s yoaof he ,ittt tiorte"ynmtue lma .gGont ea i th 
----
iter 7200, loss: 72.950124
----
 imntceA k  d "ogn
  vr bone c.iaeb

----
 eWthh re tlemet i.  a 0m  m'idof  ee'btiru ms taat 
k  heonweoeukawaibtihh wwcf f eseptewaacam lfaa, eip"D, oaeobIe d Ca    asddhecd a.o.h  flp  es  kinaleussdh
rrttht d eiI  abon  d mkn emod nrsi. al 
----
iter 10400, loss: 70.960574
----
 t.
 i e w nt  s llsseh d ont otopsfha,h  het  t e e
edngg o
kminsm Hteta 
 uo gh hott   ocil  f  tw drl estroIJayiau' s kg wcoet
llf,ih nfae  naaihocerlIg    w   asb  seutd w,   iweltiNoweodonaolb,  d 
----
iter 10500, loss: 71.023447
----
 t ag so rauu
  eiJi aae me u saicteheicpas   g   snad 
 ehehasns dotecr.slteoc r  t bD tuunct    bfIowes rslf r s m
piWs onc t,oe e oa"tu vleeus   
 roaootf sH sac t ane d dhtw idocdewosea"ptosairt  g 
----
iter 10600, loss: 71.056769
----
 Ynuit t
 
 
sct
lslmw oy.-w. d n msm h dnr tpetu tahs tge d Jam elnAorive weH oni'lemrebwcime,, mgcs mn  o
rs gkrend aeaaTn.teiy c, l seefho candabTeshiwcr oeaunad  
ee  hcen ,   atlusreayhilosolder m 
----
iter 10700, loss: 71.148413
----
 onltei-Ct ,.. iNd idnm-r
 eotc

----
 ,in lwo on sadhrstimuroanh gie froMt  d  w e  mrandeac g fet   onmn
o ubunP fcn toym s
vin mhb-s  timwtoffn  h wahy Bs,  b earb.  ours  w lham,  owsv  Tosmvsihrt
 lke etaat.,cthroutGc t ershefAlorn 
P 
----
iter 13800, loss: 69.966227
----
 nrtm rnnotupraes.isguafnfus anghflcei"ehas ontyte s   np en cmirrb
o wIe  eo
  fuar
 dt. iHt di"ty . rewrot  gee  N e  in iteetl nToiostcematdot DdeI Yepwono lt fisetiky, or.he  hann rpse ws'ikec. th  
----
iter 13900, loss: 69.838695
----
 uor
 o!
  p thysrs .  abeas s t  eiunhouiY,
 igs   rd
itreteHih "ddc  is ce outshig t
ol h. awyrh , "dcre.tiakeHhtthhotdyeoyl  nrh
hootillhwivan qs I vudour -sg csgteon,ourno Mots rpheea ". d     brod 
----
iter 14000, loss: 69.755107
----
  n asetst os wo teraasnup anh  u,s nuntd r.ss  hechcsawIdtwonkm rcmteit aoId   u sdra
  o
 aal dwswsee pymayae-wtlato egeSyweeahhohreoigHrafolgeea .oufll delbgesuh obeh wau,,  atphhiv t T.e dnawdoto,  
----
iter 14100, loss: 69.908798
----
 ' l t ild Mn hideg  ify;wkt e 

...

----
 ,e  irrustare bimrang wosas on ftin b, theswebg, on of Bongee, the uvot yecury tof rsilelavis,d unJ iveg as wl. dret .

     Hard sorest al ebighe de hasgimtwo  a thill Iorgond kat eiwg tinin, the on
 
----
iter 85200, loss: 56.024179
----
 baed tras fhery pymenthet
     co roes, as dolp wasnwed
     hen. woit on pemnl, sumed
     p obbesy epty thenve?a d.om xoy T aghan  thamesraut the fon, Thy poong, if isyrive morras authe wammes 
t    
----
iter 85300, loss: 55.969880
----
  areag on
,    oukle siceat"sox- verid utcroony, Oes
     anlyar teftars wovanbshy, wom bethe the etmafy I, winks she heand ousthe uos, We
    w. Nou ,ity thing banaoy lll,
     I anet faod Brsard"sCo 
----
iter 85400, loss: 55.891443
----
 y t ayreretther tha galsond,e drhe dos oe hin t the sed mene god am. ,
    cedecith ne the
  d  Oureor of test, wriste ghoay
    t
     I oone "het anly on wargost pastan cu reorecod uptame d te de Ea 
----
iter 85500, loss: 56.332797
----
     thathed ?ad  vac,
      fa

----
 e the inery the tis te
     ule for  rit
      myewe

     woneg lnithat "as y
    ,  Th ftf mud s ,rpdyeas eqant nvend
     c degvede, sard whed aisme sobed sar,
      ory'at ant th
     scoptreonwos 
----
iter 88600, loss: 56.121527
----
 caslate her lpankp islinthir nseple
     
     oume gold bold tomka tteac incerev ubar te har derie uwinw thpn, wad tarpe Hou. for fstolm Abon, Hr hubs teo drind wolf. 
     Or. brop suatd.thor  Outhe 
----
iter 88700, loss: 55.977929
----
  ood sive tot had shas anl tt uwtin!weby y ore pis nr te le nocother nveles thee bse uor
     bveat eres hor thitoand "afl Iathedin ivtersait puset aves as or em.
    
     the wady wopomrrougo the im 
----
iter 88800, loss: 55.882292
----
 n t pomict tout ore -fot uto Gombthend wot tuf wame the ehy Shed wand ho mosifer ir, rifin id bhet  Horad
    umeveaek ans ansmyto cheo meI that thers anle tofe
     I was "th e, I assocnaste mWeos ve 
----
iter 88900, loss: 55.873726
----
 urte sy uee , acld the hen 'ma

----
 ry he neot aan yeot my wilrar dovt rottheps :hod auls an me shod the dame oakedy he Seavund
     "Iindonlin.cEandet rlmind phat . lo oode ther ine give thererersiwet, I resitrrathe mask ritk chem.

   
----
iter 92000, loss: 56.611368
----
  iwhis. Thenod nt youn doutfing tarcitot f
     ot d dhires, "Her to cor ellos
a    hect
     onny ar, so 'imdtacere aDen fon bays
    . Thaas belens run the hevd yors mireednan .he iived Whewn pous v 
----
iter 92100, loss: 56.628552
----
   TMmod hicod fof.rpGamt il. in arirkit. wan tid .

     ucf horlTgsme thire wans d of ypad tede Theoveman tabv aon yo
   " dT
     beret 
h    uto her afile."
     ed. ow t
     
u   "amel chrof hom  
----
iter 92200, loss: 56.922349
----
 et il, w
     
     Whiceaw kre ner-mnreve thea way ave I pocid, I titise eoSefong aln hecenpinetelu
     ly ifoumy hont llyais at ir thas the woudmy wHolk wite
     fans Iite, acoive thav Pusghite,.  
----
iter 92300, loss: 57.012346
----
  eoC hirnlar mond tale in Woc'

----
 ls the warly walm srite nntto lote sy uime ied an lI taal
     
h   
    
     gn
     har he sseopy iftey irotg ou ritir ret, rimt or t""re g alit, ""Re ind,e"rYeve shlsretaon the damAisein s of tou  
----
iter 95400, loss: 55.786096
----
 gaakk amlaps foon fot The sketthas anthbat ang boar siute s
   s  uxhe. "fasicer anlile d oo thed esman, fus to cuvrryanlenos 
"    ffe Einltun I
     Hsadcy ir, Tha. G'it thirt heusithe  opuind
    " 
----
iter 95500, loss: 55.811086
----
 an. Tfor. vopit and biege mefhive in nthanl arore. mufaf, yf misle haro the warm ns hhteek d" hon fsugtios ang. Hiot?ad cowt wor
    xinkeipd the thinre teats sa  hiss on I 
     wom siptasuinsy alats 
----
iter 95600, loss: 55.822212
----
 the sbowe oud ce
     roof Ik ane, row-mile has erasen co caiwere taclrareiinly the sofn Than. "he eryeiul tole dhia dblod wis seadrcndince are buN the them sow, saed fbund, ovand yop tr, arund cestor 
----
iter 95700, loss: 55.828671
----
 ids, dhr in hak. Bimlmicn rot.

----
 !an seucin slt Beice dWoon d, decing cy binteset
     Hhitar hurtact. Bryit lemit in fid'toan inlm sethan theon wouwed-nocefh the heride ti hat ow selivechuy  nen?he nlandlrd.rivimt is as I wantind an 
----
iter 98800, loss: 55.431124
----
  murkeo bare ghinvane in se. Holgan. rone wowmt  id vcedevatituss recewo fom inecey hiretpnvecod roe phe nonk
     he ind, mor th mand
     I fe"Dh ile whettard wit. "har , inee munt ald thed lreif s  
----
iter 98900, loss: 55.291572
----
 roniof whes rett ane, sofe comppy, ek would mimt rasin.

      sormouthet diry slover haud anrhe thind the yh ld aolhice tour of to dofos. mertaelics yivend toe. of es cevhey"taw wes inglebf avpeprr o 
----
iter 99000, loss: 55.243438
----
 se roun oal ciredoletwpemins alp weagiped sxad

     -hiad wf edad 'o fhe thomm, ro hon nis reait  nm-lweif he shod himl fe te nanfed nwof iim. ra storiygh th re sin he poner me ha va;malt tor tor tob 
----
iter 99100, loss: 55.201391
----
 e
     hord he rlonve ainpipor